# MOEA tutorial

## overview of this notebook
* Introduction to many objective optimization with the ema workbench
    * dependencies
    * basic structure of the code
* Explanation of $epsilon$-archiving 
    * explanation of the default algorithm used in workbench
* Assessing convergence of many objective optimization algorithms
    * how to calculate and visualize various convergence metrics
* dealing with stochasticity of many objective optimization algorithms
    * the need for seed analysis
    * how this changes visualizing your results and assessing convergence


## Introduction
In the previous assignments, we have been using sampling to investigate the uncertainty space and the lever space. However, we can also use optimization algorithms to search through these spaces. Most often, you would use optimization to search through the lever space in order to find promising policies. However, we can also use optimization to search through the uncertainty space in order to find for example a worst case scenario. In this assignment, we are going through the basics of using the optimization functionality of the workbench. 

For optimization, the ema_workbench relies on a library called platypus-opt. *platypus-opt* is python package developed by David Hadka (http://platypus.readthedocs.io/en/latest/) for multi-objective optimization. It allows an explicit specification of the problem components (levers, objectives, constraints). The package includes several multi-objective evolutionary algorithms, therefore the users can choose the algorithm they wish to use. 

you can use pip to install it:

```
pip install platypus-opt
```

Start with importing the lake model we have used in previous weeks and connecting it to the workbench. However, we need to make one change: for each outcome of interest we need to specify whether we want to maximize or minimize it, we can use the `kind` kwarg for this. `max_P` should be minimized, while all other outcomes are to be maximized. As a further simplification for this tutorial, we are ignoring the inertia objective. We do this by not setting the `kind` kwarg. 


In [1]:
from lakemodel_function import lake_problem

from ema_workbench import (Model, RealParameter, ScalarOutcome,
                           MultiprocessingEvaluator, ema_logging,
                           Constant)

ema_logging.log_to_stderr(ema_logging.INFO)

#instantiate the model
lake_model = Model('lakeproblem', function=lake_problem)
lake_model.time_horizon = 100 # used to specify the number of timesteps

#specify uncertainties
lake_model.uncertainties = [RealParameter('mean', 0.01, 0.05),
                            RealParameter('stdev', 0.001, 0.005),
                            RealParameter('b', 0.1, 0.45),
                            RealParameter('q', 2.0, 4.5),
                            RealParameter('delta', 0.93, 0.99)]

# set levers, one for each time step
lake_model.levers = [RealParameter(f"l{i}", 0, 0.1) for i in 
                     range(lake_model.time_horizon)] # we use time_horizon here

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome('inertia'),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE)]

lake_model.constants = [Constant('alpha', 0.41),
                         Constant('reps', 150)],

Instead of using `perform_experiments`, we will be using `optimize`. The key difference is that `perform_experiments` is used for open exploration while `optimize` is used for directed search. So, `perform_experiments` reliens on creating an experimental design and executing these experiments, while `optimize` uses an MOEA to evolve a population.

There is several kwargs that we need to provide, so let's go through all:

* **algorithm**; We can specify which algorithm we want to use. The default is $\epsilon$-NSGA2, a state of the art many-objective evolutionary algorithm. We can use any of the other algorithms that come with platypus-opt, or the GenerationalBorg algorithm that comes with the workbench. For now, we won't change this.
* **nfe**; the number of function evaluations, this is to be determined by analyzing whether the algorithm has converged
* **searchover**; are we optimizing over the uncertainties or the levers? Most often we will be searching over the levers, so we don't generally need to change this. 
* **reference**; If we are searching over levers, what values should we assume for the uncertainties? Reference allows us to specify this. If searchover is set to levers, reference should be a `Scenario` or None, while if searchover is uncertainties, reference should be a `Policy` or None. In case of a None, the default values of the underlying model are unchanged
* **constraints**; see below
* **epsilons**; many state of the art MOEA's rely on a epsilon dominance. Basically, a grid is imposed on the objective space, and per grid cell a single solution is maintained. The granularity of the grid is specified through the epsilon values. Epsilon should be a list or array with a length equal to the number of outcomes. Below, we will see what the impact is of changing epsilon values.
* **convergence**; In order to track whether a MOEA has converged to the optimum solutions, we use convergence metrics. The workbench offers epsilon progress and hypervolume as two often used metrics for this. We will explore these below. 

let's start with a simple optimization using 5000 nfe, and 0.25, 0.1, and 0.1 as epsilon values. 

In [2]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[0.25, 0.1, 0.1])

Since we are dealing with 3 outcomes of interest, we can still visualize our results in a 3d scatter plot. Alternatively, we can visualize it using a so-called parallel coordinate plot. In a parallel coordinate plot, the dimensions are visualized side by side. A line connecting the dimensions is a single point in the multidimensional space. For more than 3 dimensions, parallel coordiante plots are prefered over 3d scatter plots with additional visual encodings for the other dimensions. The workbench has support for parallel coordinate plots using `ema_workbench.analysis.parcoords`

In [3]:
from mpl_toolkits.mplot3d import Axes3D  

outcomes = results.loc[:, ['max_P', 'utility', 'reliability']]

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(outcomes.max_P, outcomes.utility, outcomes.reliability)
ax.set_xlabel('max. P')
ax.set_ylabel('utility')
ax.set_zlabel('reliability')
plt.show()

In [4]:
from ema_workbench.analysis import parcoords

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same 
axes.invert_axis('max_P') 
plt.show()


As you can see, the parcoords figure is easier to interpret once you have learned how to read them. We can see a clear tradeoff between max_P and reliability on the one hand, and utility on the other. This is indicated by the crossing lines in between these respective dimensions. 

for the remainder of this tutorial, we will be using a four objective formulation of the problem. We add the intertia objective and we want to maximize it.

In [5]:
#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE)]

## Exploring alternative epsilon values

An important user specified parameter when using $\epsilon$-NSGAII is the epsilon parameter. $\epsilon$-NSGAII uses what is known as $\epsilon$ dominace rather than normal Pareto dominance. The idea is that a grid is imposed on the objective space, and that per grid cell, one solution is maintained. The figure below shows 2 different griddings impossed on a 2D objective space. As can be seen, the left hand figure uses an $\epsilon$ value of 0.1 for both the X and Y axis. The right hand figure uses 0.2 for the X-axis and 0.1 for the Y-axis.


<div>
<img src="figs/fig_epsilons_300dpi.png" width="800"/>
</div>


But which point is retained if you have multiple points within a single grid cell? The figure below shows how this works. Assume that we are minimizing on both $x$ and $y$, then the origin of the grid cell is the lower left corner. Next, we calculate the Euclidian distance from each point within the cell to the origin. We retain the point with the lowest Euclidian distance to the origin.

<div>
<img src="figs/fig_euclidian_300dpi.png" width="400"/>
</div>



So what are the consequences of using different epsilon values? Let's rerun the optimization, but with different epsilon values. Use \[0.5, 0.5, 0.5, 0.5\]

In [6]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[0.5, 0.5, 0.5, 0.5])

In [7]:
outcomes = results.loc[:, ['max_P', 'utility', 'reliability', 'inertia']]

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same 
axes.invert_axis('max_P') 
plt.show()

We see that by making our epsilons higher, we are coursening the grid, and thus are reducing the number of solutions we find. Let's test this by making our epsilons smaller. We now expect to find more solutions. Let's use \[0.125, 0.05, 0.05\]

In [8]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results = evaluator.optimize(nfe=5000, epsilons=[0.125, 0.05, 0.05, 0.05])

In [9]:
outcomes = results.loc[:, ['max_P', 'utility', 'reliability', 'inertia']]

limits = parcoords.get_limits(outcomes)
axes = parcoords.ParallelAxes(limits)
axes.plot(outcomes)

# we invert this axis so direction of desirability is the same 
axes.invert_axis('max_P') 
plt.show()

And as expected, we now have many more solutions. Selecting appropriate epsilon values is tricky. It depends on case specific concerns (with what granularity do we want to find solutions?), as well as runtime considerations. The lower the epsilon values, the more solutions will be maintained in the Pareto set. Given how MOEA's work, this slows down the optimization. 

## Assessing convergence
Next to selecting appropriate epsilon values, a second key issue is assessing convergence. In the foregoing, we have been running the MOEA for 5000 function evaluations. Is this sufficient? Has the algorithm converged? We have no idea. So, how can we add convergence assessment? 

There exist a variety of metrics for assessing convergence of MOEAs. The workbench supports epsilon progress and hypervolume. Epsilon progress measures how often a solution in a new grid cel of the epsilon gridded output space is found. Early on, solutions in new grid cells are found quite frequently. Once the algorithm starts to converge, progress becomes more difficult and thus epsilon progress starts to stabilize. Hypervolume is a measure for how much of the objective space is covered by a given set of non-dominated solutions. THe higher the hypervolume, the larger the space is that is covered by the space. Again, hypervolume will grow quickly early on and starts to stabilize once the algorithm is converging. 

So exactly what is hypervolume. In the figure below, the idea of hypervolume is explained visually. Let's assume we have two objectives, $x$ and $y$, which we both want to minimize. Then, in light grey you see a number of points which are being dominated by the points in blue. Hypervolume is based on these non dominated blue points. The orange shaded area is the hypervolume of this set, given that we measure it from the (1, 1) top right corner.

<div>
<img src="figs/fig_hypervolume_300dpi.png" width="400"/>
</div>

Since hypervolume requires specifying the objective space within which we want to calculate the volume (i.e. the (1,1) top right corner in the example), we need to know this space. Sometimes it is known a priori. For example in the lake problem, reliability is scalled between 0 and 1. In contrast, the bounds on max_P are not known up front. To help with this, we can introduce a constraint saying that max_P must be below a particular threshold.


In [10]:
from ema_workbench.em_framework.optimization import (ArchiveLogger,
                                                     EpsilonProgress)
from ema_workbench import Constraint

#specify outcomes 
lake_model.outcomes = [ScalarOutcome('max_P', kind=ScalarOutcome.MINIMIZE,
                                     expected_range=(0,5)),
                       ScalarOutcome('utility', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,2)),
                       ScalarOutcome('inertia', kind=ScalarOutcome.MAXIMIZE,
                                    expected_range=(0,1)),
                       ScalarOutcome('reliability', kind=ScalarOutcome.MAXIMIZE,
                                     expected_range=(0,1))]

convergence_metrics = [ArchiveLogger(
                        "./archives",
                        [l.name for l in lake_model.levers],
                        [o.name for o in lake_model.outcomes],
                        base_filename="tutorial.tar.gz",
                        ),
                        EpsilonProgress(),
                        ]


constraints = [Constraint("max pollution", outcome_names="max_P",
                          function=lambda x:max(0, x-5))]

In [11]:
with MultiprocessingEvaluator(lake_model) as evaluator:
    results, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                    epsilons=[0.125, 0.05, 0.01, 0.01],
                                    convergence=convergence_metrics,
                                    constraints=constraints)


Varous metrics arearchives = ArchiveLogger.load_archives(f"./archives/{i}.tar.gz") provied by platypus. For details on these metrics see *e.g.*, [Zatarain-Salazar et al (2016)](https://doi.org/10.1016/j.advwatres.2016.04.006) and [Gupta et al (2020)](https://doi.org/10.1016/j.advwatres.2020.103718) for hypervolume, generational distance and additive $\epsilon$-indicator; [Hadka and Reed (2012)](https://dl-acm-org.tudelft.idm.oclc.org/doi/pdf/10.1162/EVCO_a_00053?accessTab=true) for spacing; and [Hadka and Reed (2013)](https://ieeexplore.ieee.org/document/6793867) for $\epsilon$-pgrogress. To use these metrics, we first need to load the archives into memory. Next, these metrics need a set of platypus solutions, instead of the dataframes that the workbench has stored. Moreover, most of these metrics need a reference set. The reference set, typically, is the union of best solutions found across the seeds and next filtered using an ε-nondominated sort. All these steps are supported by the workbench as shown below.

In [12]:
from ema_workbench import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

def calculate_metrics(archives, reference_set):
    problem = to_problem(lake_model, searchover="levers")

    hv = HypervolumeMetric(reference_set, problem)
    gd = GenerationalDistanceMetric(reference_set, problem, d=1)
    ei = EpsilonIndicatorMetric(reference_set, problem)
    ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
    sm = SpacingMetric(problem)

    metrics = []
    for nfe, archive in archives.items():
        scores = {
            "generational_distance": gd.calculate(archive),
            "hypervolume": hv.calculate(archive),
            "epsilon_indicator": ei.calculate(archive),
            "inverted_gd": ig.calculate(archive),
            "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    return metrics


def plot_metrics(metrics, convergence):
    sns.set_style("white")
    fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

    ax1, ax2, ax3, ax4, ax5, ax6 = axes

    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

    ax6.set_xlabel("nfe")

    sns.despine(fig)

In [13]:
archives = ArchiveLogger.load_archives(f"./archives/tutorial.tar.gz")
reference_set = archives[max(archives.keys())] # this is the final archive

metrics = calculate_metrics(archives, reference_set)
plot_metrics(metrics, convergence)

plt.show()

If we look at the above plots, we can see that neither hypervolume, nor $\epsilon$-progress has stablized. 5000 function evaluations is clearly not sufficient. Let's go to another extreme: 100.000. What happens in this case?

In [14]:
convergence_metrics = [ArchiveLogger(
                        "./archives",
                        [l.name for l in lake_model.levers],
                        [o.name for o in lake_model.outcomes],
                        base_filename="tutorial_2.tar.gz",
                        ),
                        EpsilonProgress(),
                        ]

with MultiprocessingEvaluator(lake_model) as evaluator:
    results, convergence = evaluator.optimize(nfe=100000, searchover='levers',
                                    epsilons=[0.125, 0.05, 0.01, 0.01],
                                    convergence=convergence_metrics,
                                    constraints=constraints)


Note that calculating some of these metrics can become quite time consuming.

In [15]:
archives = ArchiveLogger.load_archives(f"./archives/tutorial_2.tar.gz")
reference_set = archives[max(archives.keys())] # this is the final archive

metrics = calculate_metrics(archives, reference_set)
plot_metrics(metrics, convergence)

plt.show()

The runtime of this analysis has been substantial. Still, looking at the convergence graphs, hypervolume has more or less stablized, while $\epsilon$-progress only starts to stablize. This could be an argument for running the algorithm even longer (say 250.000 nfe). Establising the number of NFE is generally a form of trial and error. 

# The role of stochasticity
MOEAs use stochastics in crossover and mutation. Thus, the specific set of results will vary from one run of the algorithm to the next. Analogous to how you deal with stochasticitiy in discrete event models, it is best practice to run an MOEA multiple times using a different random seed. Next, you would combine the results from the different runs into a combined pareto approximate set. 


In [16]:
results = []
convergences = []
        
with MultiprocessingEvaluator(lake_model) as evaluator:
    # we run again for 5 seeds
    for i in range(5):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                "./archives",
                [l.name for l in lake_model.levers],
                [o.name for o in lake_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            EpsilonProgress(),
        ]

        # 5000 runs is clearly way to low, given the convergence 
        # analysis above. this is only for demonstration purposes
        result, convergence = evaluator.optimize(nfe=5000, searchover='levers',
                                     epsilons=[0.125, 0.05, 0.01, 0.01],
                                     constraints=constraints,
                                     convergence=convergence_metrics,)        
 
        results.append(result)
        convergences.append(convergence)        

In [17]:
limits = pd.DataFrame([[0,0,0,0],[5,2,1,1]], columns=['max_P', 'utility', 'reliability', 'inertia'])
axes = parcoords.ParallelAxes(limits)

for i, (result, color) in enumerate(zip(results, sns.color_palette())):
    outcomes = result.loc[:, ['max_P', 'utility', 'reliability', 'inertia']]
    axes.plot(outcomes, color=color, label='results {}'.format(i))

# we invert this axis so direction of desirability is the same  
axes.invert_axis('max_P') 
axes.legend()
plt.show()

If we want to track convergence across all seeds, we have to be a bit carefull in how we define the reference set. No longer can we use the final archive as our reference set. Rather, we have to take the final archive for each set, and combine them. Evidently, some of the solutions from one seed might be dominated by solutions from another set. So, the way to generate a reference set is by doing another non-dominated sort over the set of final achives. Note that this final archive is the same as the results return from the optimization.


In [18]:
all_archives = []

for i in range(5):
    archives = ArchiveLogger.load_archives(f"./archives/{i}.tar.gz")
    all_archives.append(archives)

In [19]:
from ema_workbench.em_framework.optimization import epsilon_nondominated


problem = to_problem(lake_model, searchover="levers")
reference_set = epsilon_nondominated(results,  epsilons=[0.125, 0.05, 0.01, 0.01], problem=problem)

In [20]:
metrics_by_seed = []
for entry in all_archives:
    metrics = calculate_metrics(entry, reference_set)
    metrics_by_seed.append(metrics)

In [21]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

ax1, ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()

We can see in the above figure that the different seeds (shown in different colors) are stabilizing for some of the convergence metrics (e.g., generational distance), while for some of the harder to achieve metrics (e.g., hypervolume) no stabilization is yet taking place. So, the conclusion on the basis of these figures is that the algorithm for none of the seeds has covnerged adequately and a higher number of nfe is needed. Another thing worth nothing, with respect to hypervolume, is that the seeds at present are not converging to a similar hypervolume. If this persists even with a much higher number of function evalations, this might be indicative of search problems and warrants further investigation. This, however, is an advanced topic beyond the scope of this course.


# Closing remarks
This tutorial provided an introduction to using many objective optimization with the ema workbench. It explained some of the key concepts like assessing convergence, displaying results using parallel axis plots, and the need to perform seed analysis. In the next assignments, you will apply all these concepts to several cases.